In [1]:
from IPython.display import HTML
HTML('<h1>Finse Alpine Research Centre Weather Stations</h1>')

In [2]:
import folium

map = folium.Map(location=[60.5, 7.5], tiles='Stamen Terrain', zoom_start=7)

# Create two groups: 
# Group-1: Finse Alpine Research Centre Stations 
# Group-2: Norwegian Meteorological Service Weather Stations 

group_1 = folium.FeatureGroup(name='Finse Stations')
group_2 = folium.FeatureGroup(name='MetNo Stations')


# Add group_1 and group_2 to map
group_1.add_to(map)
group_2.add_to(map)
folium.LayerControl().add_to(map)

# Then we add Markers to group_1 or group_2 and not to map

In [3]:
# Finse Alpine Research Centre data

import urllib.request

url='https://raw.githubusercontent.com/annefou/jupyter_dashboards/gh-pages/data/Hardangervidda.geojson'
# Download the file from `url`, save it in a temporary directory and get the
# path to it (e.g. '/tmp/tmpb48zma.txt') in the `file_name` variable:
geojson_filename, headers = urllib.request.urlretrieve(url)

geojson = folium.GeoJson(
    geojson_filename,
    name='geojson'
)
features = geojson.data['features']
html_header="""
<html>
<head>
<style>
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}

td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
}

tr:nth-child(even) {
    background-color: #dddddd;
}
</style>
</head>
<body>

<table>
"""
html_end = """
</table>

</body>
</html>
"""
features = geojson.data['features']
for i in range(0,len(features)):   
    location=[features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0]]
    name = features[i]['properties']['name']
    opr = features[i]['properties']['waspmote_id']
    description = features[i]['properties']['description']
    country = features[i]['properties']['country.etc']
    html = html_header + """
    <tr>
      <th>name</th> <th>""" + name + """</th>
    </tr>
    <tr>
      <th>description</th><th>""" + description + """</th>
    </tr>
    <tr>
      <th>country.etc</th><th>""" + country + """</th>
    </tr>
    <tr>
      <th>wapsmote_id</th><th>""" + opr + """</th>
    </tr>
    """ + html_end

    iframe = folium.IFrame(html=html, width=300, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(location, popup=popup,  icon=folium.Icon(color='green',  icon='ok-sign')).add_to(group_1)
    

In [4]:
# Finse Norwegian Meteorological Centre

import requests 

# to be filled
client_id = ''

types='SensorSystem'
# request all stations within a polygon
polygon = 'POLYGON((6.9 60.35,6.9 60.7,8.16 60.7, 8.16 60.35, 6.9 60.35))'
# issue an HTTP GET request
r = requests.get(
        'https://frost.met.no/sources/v0.jsonld',
        {'types': types, 
        'geometry': polygon},
        auth=(client_id, '')
    )


for item in r.json()['data']:
        latitute=''
        longitude=''
        county=''
        municipality=''
        if 'geometry' in item:
            latitude = item['geometry']['coordinates'][1]
            longitude = item['geometry']['coordinates'][0]
            location = [latitude,longitude]
        if 'municipality' in item:
            municipality = item['municipality']
        if 'county' in item:
            county = item['county']
        html = """
      <h4>ID: """ + item['id'] + """</h4>
      <h4>Name: """ + item['name'] + """</h4>
      <h4>Latitude: """ + str(latitude) + """</h4>
      <h4>Longitude: """ + str(longitude) + """</h4>
      <h4>Municipality: """ + str(municipality) + """</h4>
      <h4>County: """ + str(county) + """</h4>
      <h4>Country: """ + str(item['country']) + """</h4>
    """ 
        iframe = folium.IFrame(html=html, width=300, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        folium.Marker(location, popup=popup,  icon=folium.Icon(color='blue',  icon='cloud')).add_to(group_2)



KeyError: 'data'

In [5]:
map

In [ ]:
map.save('../files/map_finse_metno_control.html')

In [ ]:
from pandas.io.json import json_normalize
from datetime import datetime
from beakerx import *


data = r.json()['data']
for element in data:
        value = element['validFrom']
        element['validFrom'] = datetime.strptime(value, "%Y-%M-%d")
data_metno=json_normalize(data)
data_metno

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go

# To initialize plotly for notebook usage
py.init_notebook_mode()

In [ ]:
# Get sonic temperature at Hills and prepare to plot with plotly (offline)

import requests

URL = 'http://hycamp.org/wsn/api/query/'
token = 'dcff0c629050b5492362ec28173fa3e051648cb1'
headers = {'Authorization': 'Token %s' % token}


sensor = 'ds2_temp'

# Finse station 'Hills'
station = 'Hills'
waspmote_id='023D67057C105474'
waspmote_id='667767057C10548E'
station_id = int(waspmote_id,16)
start_date='2017-12-01T00:00:00+00:00'
end_date='2018-01-01T00:00:00+00:00'
params  = {
            'limit': 100000,
            'offset': 0,
            'mote': station_id,
            'xbee': None,
            'sensor': sensor,
            'tst__gte': start_date,
            'tst__lte': end_date,
            }
r = requests.get(URL, headers=headers, params = params)

data_finse = r.json()['results']
FinseStations = json_normalize(data_finse)
# Add a new column where we convert 'epoch' (s) to a datetime 
FinseStations['timestamp'] = pd.to_datetime(FinseStations.epoch, unit='s')
data_finse_to_plot = go.Scatter(x=FinseStations.timestamp, y=FinseStations.value, marker = dict(size=10,color='blue'))


In [ ]:
# Get Data for Finsevatn and prepare to plot
# example of possible variables to retrieved:
# 'air_temperature', 'surface_air_pressure', 'relative_humidity', 'wind_speed'

referencetime = '2017-12-01/2018-01-01'

# Finsevatn station
source = 'SN25830'
variable = 'air_temperature'
# issue an HTTP GET request
rt = requests.get(
        'https://frost.met.no/observations/v0.jsonld',
        {'sources': source, 'elements': variable,'referencetime': referencetime}, 
        auth=(client_id, '')
)
if rt.status_code == 200:
    # Data available
    T = json_normalize(data=rt.json()['data'], record_path='observations', meta= ['referenceTime', 'sourceId'], errors='ignore')
else:
    print("Data not available")
    
T=pd.concat([T.drop(['level'], axis=1), T['level'].apply(pd.Series).rename(columns={'unit': 'levelUnit', 'value' : 'levelValue'})], axis=1)
T2m = T.loc[T['levelValue'] == 2]
T2m['referenceTime'] = pd.to_datetime(T2m['referenceTime'])

data_metno_to_plot = go.Scatter(x=T2m['referenceTime'], y=T2m['value'], marker = dict(size=10,color='red'))

In [ ]:
py.iplot([data_metno_to_plot, data_finse_to_plot])